# A Simple Kaggle Tutorial

In [1]:
!wget lovesnowbest.site/titanic.zip

--2018-07-12 05:24:11--  http://lovesnowbest.site/titanic.zip
Resolving lovesnowbest.site (lovesnowbest.site)... 23.91.101.102, 107.150.125.7, 23.91.96.169, ...
Connecting to lovesnowbest.site (lovesnowbest.site)|23.91.101.102|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://lovesnowbest.site/titanic.zip [following]
--2018-07-12 05:24:12--  https://lovesnowbest.site/titanic.zip
Connecting to lovesnowbest.site (lovesnowbest.site)|23.91.101.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35973 (35K) [application/zip]
Saving to: ‘titanic.zip’

titanic.zip         100%[===================>]  35.13K   230KB/s    in 0.2s    

2018-07-12 05:24:13 (230 KB/s) - ‘titanic.zip’ saved [35973/35973]



In [2]:
!unzip titanic.zip

Archive:  titanic.zip
   creating: Titanic/
  inflating: Titanic/.DS_Store       
   creating: __MACOSX/
   creating: __MACOSX/Titanic/
  inflating: __MACOSX/Titanic/._.DS_Store  
  inflating: Titanic/test.csv        
  inflating: __MACOSX/Titanic/._test.csv  
  inflating: Titanic/train.csv       
  inflating: __MACOSX/Titanic/._train.csv  


In [3]:
cd Titanic

/content/Titanic


In [0]:
# import tools you need: scikit-learn, pandas, numpy
# there are some other libraries: lightgbm, xgboost, tensorflow
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectPercentile,chi2,f_regression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

In [0]:
# read your data from files
rawData=pd.read_csv('train.csv')
testData=pd.read_csv('test.csv')

In [12]:
# Feature Engineering
# Usually based on your background knowledge or your inner view on dataset 
rawData['Family'] = rawData["Parch"] + rawData["SibSp"]
rawData['Family'].loc[rawData['Family'] > 0] = 1
rawData['Family'].loc[rawData['Family'] == 0] = 0

testData['Family'] = testData["Parch"] + testData["SibSp"]
testData['Family'].loc[testData['Family'] > 0] = 1
testData['Family'].loc[testData['Family'] == 0] = 0

rawData = rawData.drop(['SibSp','Parch'], axis=1)
testData = testData.drop(['SibSp','Parch'], axis=1)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
# First split the dataset rawData into two part: X and y(survived 0/1).
# Feature `name` seems useless for our prediction, just drop it out.
y = rawData['Survived']
X = rawData.drop(['PassengerId','Survived','Name'],axis=1)
X_test_Id = testData['PassengerId']
X_test = testData.drop(['PassengerId','Name'],axis=1)

In [14]:
# Handle with Missing Values
# first let's have a look at the distribution of missing values
total = X.isnull().sum().sort_values(ascending=False)
percent = (X.isnull().sum()/X.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total,percent],axis=1,keys=['Total','Percent'])
missing_data.head()

,Total,Percent
Cabin,687,0.771044
Age,177,0.198653
Embarked,2,0.002245
Family,0,0.000000
Fare,0,0.000000


In [0]:
# Handle with NULL value
X['Age'].fillna(X['Age'].mean(),inplace=True)
X_test['Age'].fillna(X['Age'].mean(),inplace=True)
X.fillna('UNKNOWN',inplace=True)
X_test.fillna('UNKNOWN',inplace=True)

In [23]:
X.head()

,Pclass,Sex,Age,Ticket,Fare,Cabin,Embarked,Family
0,3,male,22.0,A/5 21171,7.2500,UNKNOWN,S,1
1,1,female,38.0,PC 17599,71.2833,C85,C,1
2,3,female,26.0,STON/O2. 3101282,7.9250,UNKNOWN,S,0
3,1,female,35.0,113803,53.1000,C123,S,1
4,3,male,35.0,373450,8.0500,UNKNOWN,S,0


In [0]:
# Tackle with categorical features(also the UNKNOWN value)
dv = DictVectorizer()
X_train = dv.fit_transform(X.to_dict(orient='record'))
X_test = dv.transform(X_test.to_dict(orient='record'))

In [0]:
# select the best k percentiles features to get a better score
sp = SelectPercentile(chi2,percentile=92)
X_train_sp = sp.fit_transform(X_train,y)
X_test_sp = sp.transform(X_test)

In [0]:
rf_params = {
    'n_estimators': 800,
    'max_features' : 'sqrt',
}
rf = RandomForestClassifier(**rf_params)
rf.fit(X_train_sp,y)
predict_rf = rf.predict(X_test_sp)

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}
et = ExtraTreesClassifier(**et_params)
et.fit(X_train_sp,y)
predict_et = et.predict(X_test_sp)

# AdaBoost parameters
ada_params = {
    'n_estimators': 800,
    'learning_rate' : 0.5
}

ada = AdaBoostClassifier(**ada_params)
ada.fit(X_train_sp,y)
predict_ada = ada.predict(X_test_sp)

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 800,
     #'max_features': 0.2,
    'max_depth': 5,
    #'min_samples_leaf': 2,
}

gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_train_sp,y)
predict_gb = gb.predict(X_test_sp)

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }
svc = SVC(**svc_params)
svc.fit(X_train_sp,y)
predict_svc = svc.predict(X_test_sp)

In [0]:
# ensemble the prediction by all classifiers
# you can also use VotingClassifier in sklearn
one = np.ones(X_test.shape[0])
zero = np.ones(X_test.shape[0])
for i,j in enumerate(predict_rf):
	if j == 1:
		one[i] += 1
	else:
		zero[i] += 1
for i,j in enumerate(predict_ada):
	if j == 1:
		one[i] += 1
	else:
		zero[i] += 1
for i,j in enumerate(predict_gb):
	if j == 1:
		one[i] += 1
	else:
		zero[i] += 1
for i,j in enumerate(predict_et):
    if j == 1:
        one[i] += 1
    else:
        zero[i] += 1
for i,j in enumerate(predict_svc):
    if j == 1:
        one[i] += 1
    else:
        zero[i] += 1

predict = []
for i,j in zip(one,zero):
	if i < j:
		predict.append(0)
	else:
		predict.append(1)

predict_pd = pd.DataFrame(predict, columns=['Survived'])

In [0]:
final_predict = pd.concat([X_test_Id, predict_pd], axis=1)

In [22]:
final_predict.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [119]:
final_predict.to_csv('predict.csv', index=False)

In [120]:
from google.colab import files
files.download('predict.csv')
# BTW, upload a file to colab: `files.upload()`